In [8]:
import numpy as np
import sklearn

import pandas as pd

from sklearn import model_selection


from dask.distributed import Client, wait
import dask_cudf

from cuml.dask.ensemble import RandomForestClassifier as cumlDaskRF
from sklearn.ensemble import RandomForestClassifier as sklRF

In [16]:
from dask.distributed import progress

# Load scheduler config file (generated by dask-mpi) from home directory
client = Client(scheduler_file='/leonardo/home/usertrain/a08tra81/scheduler.json')
client

Connection method: Scheduler file,Scheduler file: /leonardo/home/usertrain/a08tra81/scheduler.json
Dashboard: http://10.128.59.29:8787/status,
Comm: tcp://10.128.59.29:8889,Workers: 0
Dashboard: http://10.128.59.29:8787/status,Total threads: 0
Started: 9 minutes ago,Total memory: 0 B


In [17]:
# Query the client for all connected workers
workers = client.has_what().keys()
n_workers = len(workers)
print(n_workers)
n_streams = 8 # Performance optimization

0


Define Parameters

In addition to the number of examples, random forest fitting performance depends heavily on the number of columns in a dataset and (especially) on the maximum depth to which trees are allowed to grow. Lower max_depth values can greatly speed up fitting, though going too low may reduce accuracy.

In [ ]:
# Data parameters
train_size = 100000
test_size = 1000
n_samples = train_size + test_size
n_features = 20

# Random Forest building parameters
max_depth = 12
n_bins = 16
n_trees = 1000

Generate Data on host

In this case, we generate data on the client (initial process) and pass it to the workers. You could also load data directly onto the workers via, for example, dask_cudf.read_csv(). See also the k-means MNMG notebook (kmeans_mnmg_demo.ipynb) for an alternative method of generating data on the worker nodes.

In [ ]:
X, y = datasets.make_classification(n_samples=n_samples, n_features=n_features,
                                 n_clusters_per_class=1, n_informative=int(n_features / 3),
                                 random_state=123, n_classes=5)
X = X.astype(np.float32)
y = y.astype(np.int32)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=test_size)

In [ ]:
n_partitions = n_workers

def distribute(X, y):
    # First convert to cudf (with real data, you would likely load in cuDF format to start)
    X_cudf = cudf.DataFrame(X)
    y_cudf = cudf.Series(y)

    # Partition with Dask
    # In this case, each worker will train on 1/n_partitions fraction of the data
    X_dask = dask_cudf.from_cudf(X_cudf, npartitions=n_partitions)
    y_dask = dask_cudf.from_cudf(y_cudf, npartitions=n_partitions)

    # Persist to cache the data in active memory
    X_dask, y_dask = \
      dask_utils.persist_across_workers(c, [X_dask, y_dask], workers=workers)
    
    return X_dask, y_dask

X_train_dask, y_train_dask = distribute(X_train, y_train)
X_test_dask, y_test_dask = distribute(X_test, y_test)

In [ ]:
%%time

# Use all avilable CPU cores
skl_model = sklRF(max_depth=max_depth, n_estimators=n_trees, n_jobs=-1)
skl_model.fit(X_train.get(), y_train.get())

In [ ]:
%%time

cuml_model = cumlDaskRF(max_depth=max_depth, n_estimators=n_trees, n_bins=n_bins, n_streams=n_streams)
cuml_model.fit(X_train_dask, y_train_dask)

wait(cuml_model.rfs) # Allow asynchronous training tasks to finish

In [ ]:
skl_y_pred = skl_model.predict(X_test.get())
cuml_y_pred = cuml_model.predict(X_test_dask).compute().to_numpy()

# Due to randomness in the algorithm, you may see slight variation in accuracies
print("SKLearn accuracy:  ", accuracy_score(y_test, skl_y_pred))
print("CuML accuracy:     ", accuracy_score(y_test, cuml_y_pred))